In [1]:
import cv2
import numpy as np 
import pandas as pd 
import os
from PIL import Image
from tensorflow.keras.applications.resnet50 import ResNet50
import tensorflow as tf
import tensorflow.keras as K
from tensorflow.keras import datasets, layers, models, losses, Model
from sklearn.model_selection import train_test_split
os.environ["CUDA_VISIBLE_DEVICES"] = '2,3'


2023-06-26 17:22:02.673547: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-26 17:22:02.794288: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-06-26 17:22:03.234005: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.2/lib64:/usr/local/cuda-11.6/lib64:/home/gil/anaconda3/envs/LeeYS/lib/
2023-06-26 17:22:03.234119: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library

In [3]:
input_img_size = (512,512, 3)
pd_img_list=pd.read_csv('../../data/preprocessing_dataset/data_set.csv')['file_path']
img_list=pd_img_list.to_list()
pd_label_list=pd.read_csv('../../data/preprocessing_dataset/data_set.csv')['standard_frame']
label_list=pd_label_list.to_list()

In [5]:
size=512
x = np.zeros((len(img_list),size,size,3))
y = np.zeros((len(label_list),1))
for i in range(len(img_list)):
    x[i] =np.array(Image.open('../../data/preprocessing_dataset/'+img_list[i]).resize((size,size)))
y=np.array(label_list)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)


In [7]:

checkpoint_filepath = "../../model/resnet50_preprocessing_checkpoints_{epoch:03d}.tf"
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only= True
)
class_weight = {"standard_frame": 0.15,
                "not_standard_frame": 0.85}
mirrored_strategy = tf.distribute.MirroredStrategy(devices=['/GPU:0','/GPU:1'])
with mirrored_strategy.scope(): 
    input_t=K.Input(shape=(size,size, 3))
    input_tensor = layers.experimental.preprocessing.Resizing(size, size, interpolation="bilinear", input_shape=x_train.shape[1:])(input_t)
    ResNet=ResNet50(include_top=True,weights='imagenet',input_tensor=input_tensor)
    model = K.models.Sequential()
    model.add(ResNet)
    model.add(tf.keras.layers.Dropout(.2, input_shape=(64,)))
    model.add(K.layers.Dense(64, activation=tf.keras.layers.LeakyReLU(alpha=0.1)))
    model.add(K.layers.Dense(1, activation='sigmoid'))
    model.compile(optimizer=K.optimizers.Adam(lr=1e-3),
                  loss=tf.keras.losses.binary_crossentropy,
                  metrics=["accuracy"])
    histo=model.fit(
        x_train,y_train,
        validation_data=(x_test,y_test),
        epochs=500,
        callbacks=[model_checkpoint_callback],
        batch_size=16,shuffle=True,
        class_weight=class_weight
    )
model.save('../../model/ResNet50_preprocessing512.h5')

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')


/home/gil/anaconda3/envs/LeeYS/lib/python3.9/site-packages/keras/optimizers/optimizer_v2/adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)
